<a href="https://colab.research.google.com/github/nerobite/neural_networks/blob/main/Collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-02-05 06:08:26--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 173.194.212.139, 173.194.212.138, 173.194.212.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.212.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-02-05 06:08:26--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.216.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.008s  

2024-02-05 06:08:26 (99.7 M

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162670 sha256=d152783cce657501cf2e36153d2adc6b3bda5617da09f932e31d3011719ec678
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [ ]:
ratings.rating.min()

0.5

In [ ]:
ratings.rating.max()

5.0

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

In [ ]:
algo = SVD(n_factors=200, n_epochs=30, biased=True, init_mean=0, init_std_dev=0.05, lr_all=0.006, reg_all=0.03)

In [ ]:
from surprise.model_selection import cross_validate

In [ ]:
cv_results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8487  0.8528  0.8595  0.8520  0.8476  0.8521  0.0042  
Fit time          4.25    3.97    4.83    3.99    4.00    4.21    0.33    
Test time         0.36    0.15    0.15    0.32    0.20    0.24    0.09    


In [ ]:
print(f"Средняя оценка RMSE на 5 фолдах: {np.mean(cv_results['test_rmse'])}")

Средняя оценка RMSE на 5 фолдах: 0.8521069387023286


In [ ]:
from surprise.model_selection import GridSearchCV

In [ ]:
param_grid = {'n_factors': [100, 150, 200], 'n_epochs':[20, 30, 40], 'init_std_dev':[0.03, 0.05, 0.07], 'lr_all':[0.004,0.006, 0.008], 'reg_all': [0.01, 0.03, 0.05]}

In [ ]:
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)

In [ ]:
print(f"Наилучшие параметры: {grid_search.best_params['rmse']}")
print(f"Лучшая оценка RMSE: {grid_search.best_score['rmse']}")

Наилучшие параметры: {'n_factors': 200, 'n_epochs': 40, 'init_std_dev': 0.03, 'lr_all': 0.008, 'reg_all': 0.05}
Лучшая оценка RMSE: 0.8536366170139229


In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
accuracy.rmse(predictions, verbose=True)

RMSE: 0.8495


0.8495403659722592

In [ ]:
from surprise import KNNBaseline

In [ ]:
algo_knn = KNNBaseline(k=100, min_k=5)

In [ ]:
algo_knn.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
predictions_knn = algo_knn.test(testset)

In [ ]:
accuracy.rmse(predictions_knn, verbose=True)

RMSE: 0.8647


0.8647027654485294

In [ ]:
from surprise import SlopeOne

In [ ]:
algo_s = SlopeOne()

In [ ]:
algo_s.fit(trainset)

In [ ]:
predictions_s = algo_s.test(testset)

In [ ]:
accuracy.rmse(predictions_s, verbose=True)

RMSE: 0.8946


0.8945783911352443

In [ ]:
from surprise import NMF

In [ ]:
algo_nmf = NMF(n_factors=80, n_epochs=150, biased=False, reg_pu=0.2, reg_qi=0.07, reg_bu=0.05, reg_bi=0.02, lr_bu=0.007, lr_bi=0.007, init_low=0, init_high=1, random_state=0, verbose=False)

In [ ]:
algo_nmf.fit(trainset)

In [ ]:
predictions_nmf = algo_nmf.test(testset)

In [ ]:
accuracy.rmse(predictions_nmf, verbose=True)

RMSE: 0.8792


0.8791827644595035